In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

model = "mistral-large-latest"

api_key = 'WbwKVXrrMHUqV5BWUzUB5YBWjOGEUWWw'
client = MistralClient(api_key=api_key)

def chat_mistral(prompt: str):
   messages = [
       ChatMessage(role="user", content=prompt)
   ]

   # No streaming
   chat_response = client.chat(
       model=model,
       messages=messages,
   )

   return chat_response.choices[0].message.content

In [ ]:
import json

def get_mistral_response(argobot_answer, question, context):
    prompt = '''return a single number in json format. Do not return anything except json'''
    prompt = f'''
        # Introduction
        You are a specialist RAG evaluation.
        Your task is to evaluate another chatbot's answer.
        
        # Instruction
        After receiving the another chatbot's answer, you must return 0 if the answer is not in the context, and 1 vice versa.
        Keep in mind. Argobot answer and context is in Vietnamese, so you should translate it to English before evaluation.
        You can also return 1 if:
            - If the question is irrelevant to the context.
            - If the chatbot answer is no because of the lack of knowledge, and the context actually does not contain the answer for the question.
        Don't be too harsh. A question is relevant if it contains the some keywords in the context.
        
        Answer is: <argobot_answer>{argobot_answer} <argobot_answer>
        Question is: <question>{question} <question>
        Context is: <context>{context}<context>
        
        You must respond in the following structured JSON format, all fields are mandatory:
        {{
            "score": 0 or 1 based on the relevance of answer to the context
        }}
    '''
    score_json = chat_mistral(prompt)
    score = json.loads(score_json)['score']
    return score
# tmp = get_mistral_response(
#     argobot_answer='A chicken is black',
#     context='Some chickens are yellow, but the rest are red'
# )
# print(tmp)

In [ ]:
import requests

def get_chatbot_answer(question):
    url = 'http://localhost:8000/api/v1/chat/generateAnswer'
    headers = {
        'accept': 'application/json'
    }
    data = {
        'session_id': '1',
        'sender_message': question
    }
    
    files = {
        'file': None
    }

    response = requests.post(url, headers=headers, data=data, files=files)
    
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        return data['response']
    else:
        print(f"Failed to retrieve data: {response.status_code}")

In [ ]:
import pandas as pd

dataset = pd.read_excel('kb_data.xlsx')
dataset.head()

In [ ]:
#evaluate
total_score = 0
for i in range(len(dataset)):
    question = dataset.iloc[i]['question']
    context = dataset.iloc[i]['content']
    
    answer = get_chatbot_answer(question)
    score = get_mistral_response(answer, question, context)
    total_score += score
    print(i, score, '\n', question, '\n', answer)
print(total_score, '/', len(dataset))